In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from PIL import Image
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import confusion_matrix
from skimage.transform import resize
import itertools
import os
import shutil
import random
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
def loadData():
    # Load training data
    train_dataset = h5py.File('/cxldata/datasets/project/cat-non-cat/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels
    
    # Load test data
    test_dataset = h5py.File('/cxldata/datasets/project/cat-non-cat/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels
    
    # Check all the classes
    classes = np.array(test_dataset["list_classes"][:])
    
    # Reshape the train and test set labels
    train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes

In [ ]:
XTrain, yTrain, XTest, yTest, classes = loadData()

In [ ]:
print("X train shape", XTrain.shape)
print("X test shape", XTest.shape)
print("y train shape", yTrain.shape)
print("y test shape", yTest.shape)

In [ ]:
# Example of a picture

index = 50
plt.imshow(XTrain[index]) 
y = yTrain[:, index]
y_class = classes[np.squeeze(yTrain[:, index])].decode("utf-8")
print(y)
print(y_class)

Some helper functions

In [ ]:
# Preprocess for VGG-16 - normalise color chanels & resize
def preprocessImage(x):
    preprocessedImgs = tf.keras.applications.vgg16.preprocess_input(
        x, data_format=None)
    new_shape = (224, 224)
    resizedImgs = np.array([resize(image, new_shape) for image in preprocessedImgs])
    return resizedImgs

In [ ]:
# Plot images
def plotImages(x, y, index):
    plt.imshow(x[index]) 
    y = y[index,:]
    print(y)

In [ ]:
# Confusion Matric sklearn code
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Data preprocessing

In [ ]:
Xtrain_pp = preprocessImage(XTrain)
Xtest_pp = preprocessImage(XTest)

In [ ]:
type(Xtrain_pp)

In [ ]:
Xtrain_pp.shape

In [ ]:
yTrain.shape

In [ ]:
yTrain = yTrain.reshape(-1,1)
yTrain.shape

In [ ]:
yTest = yTest.reshape(-1,1)
yTest.shape

In [ ]:
plotImages(Xtrain_pp, yTrain, 50)

In [ ]:
plotImages(Xtrain_pp, yTrain, 10)

Build a CNN

In [ ]:
cnnModel = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])

In [ ]:
cnnModel.summary()

In [ ]:
cnnModel.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
cnnModel.fit(Xtrain_pp, yTrain, validation_split=0.15, epochs=20, verbose=2)

# TO DO - plot training curves

Results on training data

In [ ]:
predictions = cnnModel.predict(Xtrain_pp, verbose=0)

cm = confusion_matrix(y_true=yTrain, y_pred=np.argmax(predictions, axis=-1))

cm_plot_labels = ['not-cat','cat']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

Results on test data

In [ ]:
predictions = cnnModel.predict(Xtest_pp, verbose=0)

cm = confusion_matrix(y_true=yTest, y_pred=np.argmax(predictions, axis=-1))

cm_plot_labels = ['not-cat','cat']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

Use transfer learning to build the model using VGG-16

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
vgg16_model.summary()

In [ ]:
model = Sequential()

for layer in vgg16_model.layers[:-1]:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(Xtrain_pp, yTrain, validation_split=0.1, epochs=10, verbose=2)

Results on training Data

In [ ]:
predictions = model.predict(Xtrain_pp, verbose=0)

cm = confusion_matrix(y_true=yTrain, y_pred=np.argmax(predictions, axis=-1))

cm_plot_labels = ['not-cat','cat']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

Results on test data

In [ ]:
predictions = model.predict(Xtest_pp, verbose=0)

cm = confusion_matrix(y_true=yTest, y_pred=np.argmax(predictions, axis=-1))

cm_plot_labels = ['not-cat','cat']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')